In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv1D,
    MaxPooling1D,
    LSTM,
    Bidirectional,
    Dropout,
    Flatten,
    Dense,
    Input,
)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session

from utils import load_dataset, train, plot_roc, auc
from einops import rearrange
import matplotlib.pyplot as plt
import numpy as np

[Reference](https://github.com/uci-cbcl/DanQ/blob/master/DanQ_train.py) 

In [2]:
clear_session()
model = Sequential(
    [
        Input((1000, 4)),
        Conv1D(320, kernel_size=26, activation="relu"),
        MaxPooling1D(pool_size=13, strides=13),
        Dropout(0.2),
        Bidirectional(LSTM(160, return_sequences=True)),
        Dropout(0.5),
        Flatten(),
        Dense(10*640, activation="relu"),
        Dense(31, activation="sigmoid"),
    ],
    name="danq_FOX",
)
model.summary()

Model: "danq_FOX"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 975, 320)          33600     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 75, 320)           0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 320)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 75, 640)           1640960   
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 640)           0         
_________________________________________________________________
flatten (Flatten)            (None, 48000)             0         
_________________________________________________________________
dense (Dense)                (None, 6400)              307

In [3]:
X_train, y_train, X_test, y_test = load_dataset(
    file="FOX_3000",
    out_dir="/home/victor/Documents/datasets",
    labels="binlabels"
)

X_train = rearrange(X_train, "w h c -> w c h")
X_test = rearrange(X_test, "w h c -> w c h")

# y_train = np.where(y_train < boundary, 0, 1)
# y_test = np.where(y_test < boundary, 0, 1)

In [4]:
es = EarlyStopping(monitor="loss", patience=10)
optimizer = Adam(lr=1e-3)

model.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=["acc"],
)

In [5]:
model = train(
    dataset=(X_train, y_train, X_test, y_test),
    model=model,
    epochs=60,
    validation_freq=1,
    callbacks=[es],
    verbose=1,
    batch_size=100,
)

# y_pred = model.predict(X_test)
# plot_roc(y_test, y_pred)
# print(auc(y_test, y_pred))

Epoch 1/60
10000/10000 [==============================] - 1427s 142ms/step - loss: 0.1769 - acc: 0.1414 - val_loss: 0.1443 - val_acc: 0.2874
Epoch 2/60
10000/10000 [==============================] - 1428s 143ms/step - loss: 0.1470 - acc: 0.2827 - val_loss: 0.1421 - val_acc: 0.2979
Epoch 3/60
10000/10000 [==============================] - 1428s 143ms/step - loss: 0.1443 - acc: 0.2905 - val_loss: 0.1397 - val_acc: 0.3022
Epoch 4/60
10000/10000 [==============================] - 1429s 143ms/step - loss: 0.1427 - acc: 0.2948 - val_loss: 0.1397 - val_acc: 0.3046
Epoch 5/60
10000/10000 [==============================] - 1433s 143ms/step - loss: 0.1410 - acc: 0.3013 - val_loss: 0.1389 - val_acc: 0.3059
Epoch 6/60
10000/10000 [==============================] - 1433s 143ms/step - loss: 0.1397 - acc: 0.3044 - val_loss: 0.1396 - val_acc: 0.3036
Epoch 7/60
10000/10000 [==============================] - 1435s 144ms/step - loss: 0.1382 - acc: 0.3099 - val_loss: 0.1391 - val_acc: 0.3044
Epoch 8/60
10

KeyboardInterrupt: 

model.save(f"../model_saves/evals/{model.name}-{n}000.h5")